In [1]:
#!pip install rdflib
import rdflib
from rdflib import Graph
import pandas as pd
import copy
import json
import urllib.parse

In [ ]:
g = Graph()
g.parse("../FINAL/kg-microbe_wo_dupes.ttl")

In [4]:
print(len(g))

6827261


In [5]:
q = """
SELECT ?s ?p ?o
{
    ?s ?p ?o
}
LIMIT 10
"""

gq = g.query(q)

for row in gq:
    print(f"sub: {row.s} pred: {row.p} obj: {row.o}")

sub: http://purl.obolibrary.org/obo/NCBITaxon_432608 pred: https://w3id.org/biolink/vocab/synonym obj: Candidatus Mycoplasma haemobos
sub: urn:unknown:strain%3ACIP-52.72 pred: http://www.w3.org/1999/02/22-rdf-syntax-ns#type obj: https://w3id.org/biolink/vocab/OrganismTaxon
sub: http://purl.obolibrary.org/obo/NCBITaxon_1543864 pred: http://purl.org/dc/terms/identifier obj: http://purl.obolibrary.org/obo/NCBITaxon_1543864
sub: http://purl.obolibrary.org/obo/CHEBI_112369 pred: https://w3id.org/biolink/vocab/subclass_of obj: http://purl.obolibrary.org/obo/CHEBI_51681
sub: http://purl.obolibrary.org/obo/CHEBI_143237 pred: https://w3id.org/biolink/vocab/subclass_of obj: http://purl.obolibrary.org/obo/CHEBI_37919
sub: http://purl.obolibrary.org/obo/NCBITaxon_566547 pred: https://w3id.org/biolink/vocab/location_of obj: urn:unknown:isolation_source%3Ahost_animal_endotherm_intratissue
sub: https://www.rhea-db.org/rhea/20282 pred: https://w3id.org/biolink/vocab/has_input obj: http://purl.obolibra

In [10]:
df = pd.read_csv("C:/Users/Dominika/Documents/SKOLA/ING/DIPLOMKA/output/test_data.tsv", delimiter = "\t")
df

,subject,CHEBI:100147,CHEBI:12931,CHEBI:12936,CHEBI:132943,CHEBI:132950,CHEBI:133748,CHEBI:13705,CHEBI:143136,CHEBI:14314,...,temp_range:very_low,temperature:mesophilic,temperature:psychrophilic,temperature:thermophilic,trophic_type:chemoheterotrophy,trophic_type:chemoorganoheterotrophy,trophic_type:chemoorganotrophy,trophic_type:chemotrophy,trophic_type:heterotrophy,trophic_type:organotrophy
0,NCBITaxon:1122600,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,NCBITaxon:684552,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,NCBITaxon:452922,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
3,NCBITaxon:447421,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,NCBITaxon:273384,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1226,NCBITaxon:1465756,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1227,NCBITaxon:872983,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1228,NCBITaxon:184924,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1229,NCBITaxon:1223526,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [15]:
pairs = []
for i in range(len(df)): 
    for j in range (1, len(df.columns)):
        if df.iloc[i, j] == 1:
            sub = df.iloc[i, 0]
            val = df.columns[j] 
            pairs.append([sub, val])

In [16]:
len(pairs)

28116

In [17]:
taxons = []
for i in range(len(df)): 
    taxons.append(df.iloc[i, 0])

In [18]:
len(taxons)

1231

In [22]:

bioregistry_prefixes = {}
f = open("C:/Users/Dominika/Documents/SKOLA/ING/DIPLOMKA/registry.json")
data = json.load(f)
for entity in data.values():
    bioregistry_prefixes[entity["prefix"]] = {"name": entity["name"], "uri_format": entity["uri_format"]}
f.close()
del data



def extract_uri(id: str):
    uri_parts = id.split(":", 1)
    if len(uri_parts) == 1:
        encoded_id = urllib.parse.quote(id, safe="")
        return f"urn:unknown:{encoded_id}"
    elif len(uri_parts) > 1:
        prefix = uri_parts[0].lower()
        def_prefix = bioregistry_prefixes.get(prefix)
        if def_prefix is None:
            encoded_id = urllib.parse.quote(id, safe="")
            return f"urn:unknown:{encoded_id}"
        else:
            uf = def_prefix["uri_format"]  # type: str
            if uf is None or "$1" not in uf:
                return f"https://bioregistry.io/{prefix}:{uri_parts[1]}"
            else:
                return uf.replace("$1", uri_parts[1])
    return None


taxons_prefixed = []
for row in taxons:
    uri_1 = extract_uri(row)
    taxons_prefixed.append(uri_1)

len(taxons_prefixed)

1231

In [23]:
query = """
SELECT ?s ?p ?o
WHERE {
  ?s ?p ?o .
}
"""
new_graph = Graph()

for row in taxons_prefixed:
    subjectitos = rdflib.term.URIRef(row)
    filtered_gq = g.query(query, initBindings = {
        "s": subjectitos
    })
    new_graph += filtered_gq

print(len(new_graph))

query = """
SELECT ?s ?p ?o
WHERE {
  ?s ?p ?o .
}
"""

for row in taxons_prefixed:
    objectitos = rdflib.term.URIRef(row)
    filtered_gq = g.query(query, initBindings = {
        "o": objectitos
    })
    new_graph += filtered_gq

print(len(new_graph))

39049
50703


In [24]:
new_real_graph = Graph()
for row in new_graph:
    new_real_graph.add(row)

len(new_real_graph)

50703

In [25]:
for row in new_real_graph:
    g.remove(row)

In [29]:
len(g)

1924792

In [22]:
g.serialize(destination="train_data_from_spaqrl.ttl")

<Graph identifier=N12ce4aa801664865a5415eb784c722cb (<class 'rdflib.graph.Graph'>)>

In [28]:
new_g = Graph()
new_g.parse("../FINAL/kg-microbe_wo_dupes.ttl")

KeyboardInterrupt: 

In [ ]:
q = """
SELECT ?s ?p ?o
{
    ?s ?p ?o
}
"""

gq = g.query(q)

train_taxons = []

for row in gq:
    if (row.s.startswith("obo:NCBITaxon")):
        train_taxons.append(row.s)

print(len(train_taxons))

In [ ]:
query = """
SELECT ?s ?p ?o
WHERE {
  ?s ?p ?o .
}
"""
new_graph_train = Graph()

for row in train_taxons:
    subjectitos = rdflib.term.URIRef(row)
    filtered_gq = g.query(query, initBindings = {
        "s": subjectitos
    })
    new_graph_train += filtered_gq

print(len(new_graph_train))

query = """
SELECT ?s ?p ?o
WHERE {
  ?s ?p ?o .
}
"""

for row in train_taxons:
    objectitos = rdflib.term.URIRef(row)
    filtered_gq = g.query(query, initBindings = {
        "o": objectitos
    })
    new_graph_train += filtered_gq

print(len(new_graph_train))

In [ ]:
new_real_graph_train = Graph()
for row in new_graph_train:
    new_real_graph_train.add(row)

len(new_real_graph_train)

In [ ]:
for row in new_real_graph_train:
    new_g.remove(row)

In [27]:
new_g.serialize(destination="test_data_from_spaqrl.ttl")

<Graph identifier=N862500ad7f7c4f69bc9ab88e9bc56189 (<class 'rdflib.graph.Graph'>)>